## Dependencies

In [3]:
# Confirmed needed dependencies
import pprint
import pandas as pd
import requests
import json

# Dependencies for geocoordinates generator
import sys
import random
import math
import gmplot

# Dependencies for conversion of coordinates to addresses
import geopy
from geopy.geocoders import Nominatim
import time

# Dependencies for Zillow data
from pyzillow.pyzillow import ZillowWrapper, GetDeepSearchResults, GetUpdatedPropertyDetails

# Dependency for Heat Mapper
import gmaps


# Add config.py file with the following variables and cooresponding Zillow API keys
from config import Zapi, Zapi01, Zapi02, Zapi03, Zapi04, Zapi05, Zapi06, Zapi07, Zapi08, Zapi09, Zapi10, Zapi11, Zapi12, Zapi13, Zapi14, Zapi15, Zapi16, Zapi17, Zapi18, Zapi19, Zapi20, Ztroy1, Ztroy2, Ztroy3, Zseth, Zseth2, Zkat, Zval, Zyuta
# from config import google_API_Key
# from config import census_API_Key

################# ONGOING EDITS TO REQUIREMENTS.MD #################
###### IF ANY ERRORS OCCUR RELATING TO MODULES OR CONFIG.PY #######
### REFER TO requirements.md TO ENSURE YOU ARE PROPERLY SETUP ####

ModuleNotFoundError: No module named 'config'

## File inputs/outputs

In [5]:
masterDataCLEAN_csv = "./Data/masterDataCLEAN.csv"


randLatLon_csv = "./Data/randomLatLon.csv" 
addressList_csv = "./Data/addressList.csv"
masterData_csv = "./Data/masterData.csv"
masterDFIMPORTclean_csv = "./Data/masterDFIMPORTclean.csv"
masterData100_csv = "./Data/masterData100.csv"
masterData1000_csv = "./Data/masterData1000.csv"

## Helpful Functions

In [4]:
#############################
##### VALERIE'S BLOCKS #####
###########################

# Funtion for reading CSV in as DataFrame
def csvDF(oldCSVfilepath):
    csvIN = pd.read_csv(oldCSVfilepath)
    DF = pd.DataFrame(csvIN)
    return DF

# Function for converting DataFrame to CSV
def DFcsv(dataframe, newCSVfilepath):
    dataframe.to_csv(newCSVfilepath, index=False, header=True)
    print(f"Successfully written to '{newCSVfilepath}'")
    
# Function for reading in csv, checking for headers, and appending if appropriate
def csvDFappend(oldCSVfilepath, newColumn):
    csvIN = pd.read_csv(oldCSVfilepath)
    DF = pd.DataFrame(csvIN)
    # Checking to ensure new header name does not match any current headers
    colNames = DF.columns
    for value in colNames:
        if value == newColumn:
            print("Cannot append column that matches an existing column name")
            return DF
    # Check to ensure length of newColumn matches length of current dataframe columns
    if len(newColumn) != len(DF):
        print("Cannot append column that is not the same length as existing dataframe")
        return DF
    # Append newColumn to Dataframe
    DF[newColumn] = newColumn
    return DF

## Geocoordinates of Austin

In [ ]:
##########################################
# this section written by troy bailey.   #
# enter uservariables below to determine #
# center location, radius of circle, and #
# number of geocoordinates to generate.  #
##########################################

In [ ]:
########################
#### USER VARIABLES ####
########################

x0 = 30.27444       #### Set center coordiantes in decimal degrees
y0 = -97.74028      #### initial coordiantes are location of Texas State Capitol Building

radius = 20         #### Set radius in miles

points = 40000        #### Set number of lat,lon points to generate

In [ ]:
# variables and inputs for coordinate calculations
lat_lon_list = []
radiusInDegrees=radius/69           
r = radiusInDegrees
points += 1

In [ ]:
# calculate each coordiante point and build a list of lat and lon
for i in range(1, points):
    u = float(random.uniform(0.0,1.0)) #random number for radius length
    v = float(random.uniform(0.0,1.0)) #random number for pi radians
    
    w = r * math.sqrt(u) #radius length
    t = 2 * math.pi * v  #radians
    x = w * math.cos(t)  #calculate x coord distance
    y = w * math.sin(t)  #calculate y coord distance
    
    xLat  = x + x0       #offset x by center x
    yLon = y + y0        #offset y by center y
    
    lat_lon_list.append([xLat,yLon])

# convert list to dataframe
lat_lon_df = pd.DataFrame(lat_lon_list, columns=['lat','lon'])

lat_lon_df.head()

len(lat_lon_df)

In [ ]:
# write a CSV file of coordinate points
lat_lon_df.to_csv(randLatLon_csv, index=False, header=True)

## Plot coordinate points on map


In [16]:
# This cell by Troy
# This section will plot points on a Google map centered at centerPointLat and centerPointLon with a magnification of magFactor
# It assumes there is a dataframe with "lat" and "lon" columns
# The resulting map is saved to a file called 

lat_lon_df = pd.read_csv("./Data/Archived/randomLatLon.csv")
centerPointLat = 30.27444  #these are the coordinates of the Texas State Capitol building
centerPointLon = -97.74028 #these are the coordinates of the Texas State Capitol building
magnificationFactor = 10
pointColor = "red"
pointSize = 100
mapOutputFile = "randLatLonMap.html"
df = lat_lon_df

gmap = gmplot.GoogleMapPlotter(centerPointLat, centerPointLon, magnificationFactor)

gmap.scatter(df["lat"], df["lon"], pointColor, size=pointSize, marker=False)

gmap.draw("./Presentation/" + mapOutputFile)

len(df)

40000

## Convert Coordinates to Residential Addresses

In [ ]:
#########################
##### Yuta's Blocks #####
#########################

##### Geopy Nominatim API #####
geopy.geocoders.options.default_user_agent = "ut-group-EPIC"

pp = pprint.PrettyPrinter(indent=4)

url = "https://nominatim.openstreetmap.org/reverse?"

In [ ]:
# Test API - Known Residential Address
params_1 = {
    "format": "jsonv2",
    "lat": 30.440777,
    "lon": -97.777048
}

print("===== Test Home Response:")
response = requests.get(url, params=params_1).json()
pp.pprint(response)
print("\n" + "="*60 + "\n")

In [ ]:
# Import CSV, put into DataFrame
latlon_df = pd.read_csv(randLatLon_csv)
latlon_df.head()

In [ ]:
# Put latitudes and longitudes into a zip object
lats = latlon_df.iloc[:, 0]
lons = latlon_df.iloc[:, 1]
lat_lons = []
lat_lons = zip(lats, lons)

In [ ]:
##### Loop Request API for Addresses / Append to lists #####
# Make sure to import time

query_url = "https://nominatim.openstreetmap.org/reverse?"

house_num = []
road = []
postcode = []
lat = []
lon = []
neighborhood = []

counter = 1
numRequests = latlon_df["lat"].count()
rSuccess = []
rFailure = []

print(f"Processing {numRequests} Requests...")

# Nominatim API Request

for lat_lon in lat_lons:
    params = {
        "format": "jsonv2",
        "lat": lat_lon[0],
        "lon": lat_lon[1]
    }

    time.sleep(1.1)
    response = requests.get(query_url, params=params).json()

    if response['type'] == 'house' or response['type'] == 'yes':
        lat.append(response['lat'])
        lon.append(response['lon'])
        
        try:
            postcode.append(response['address']['postcode'])
        except (KeyError, IndexError):
            postcode.append("NA")
        try:
            house_num.append(response['address']['house_number'])
        except (KeyError, IndexError):
            house_num.append("NA")
        try:
            road.append(response['address']['road'])
        except (KeyError, IndexError):
            road.append("NA")
        try:
            neighborhood.append(response['address']['neighbourhood'])
        except (KeyError, IndexError):
            neighborhood.append("NA")
        
        print(f"Processed Record {counter} of {numRequests}.")
        rSuccess.append(counter)
        counter += 1
        
    else:
        print(f"Wrong Type - Skipped Record {counter} of {numRequests}.")
        rFailure.append(counter)
        counter += 1
        
print(f"Finished Requests !!!")

In [ ]:
print("Request Results:")
print("Success #:" + str(len(rSuccess)))
print("Skipped #:" + str(len(rFailure)))

In [ ]:
# Create dataframe with addresses from API requests
address_df = pd.DataFrame({
    "house #": house_num,
    "street": road,
    "zipcode": postcode,
    "lat": lat,
    "lon": lon,
    "neighborhood": neighborhood,
})

# Clean up Dataframe Columns before output (Drop incomplete zipcodes, Highway streets, and Null house # or streets)
address_df = address_df[address_df['zipcode'].str.len() == 5]
address_df = address_df[address_df['zipcode'].apply(lambda x: len(str(x)) > 3)]
address_df = address_df[address_df['street'].str.contains("Highway") == False]
address_df = address_df[address_df['house #'].str.contains("NA") == False]
address_df = address_df[address_df['street'].str.contains("NA") == False]
address_df.dtypes

In [ ]:
# write a CSV file of addresses
address_df.to_csv(addressList_csv, index=False, header=True)

In [ ]:
# Map out CSV with gmplot

addressList_csv = "./Data/addressList.csv"

gmap = gmplot.GoogleMapPlotter(30.27444, -97.74028, 10)

gmap.scatter(addressList_csv["lat"], addressList_csv["lon"], 'red', size=20, marker=False)

gmap.draw("./Visuals/myaddressmap.html")

## Plot Addresses on a Map

In [19]:
# This cell by Troy
# This section will plot points on a Google map centered at centerPointLat and centerPointLon with a magnification of magFactor
# It plots the addresses we have selected from the random Lat Lon points
# The resulting map is saved to a file called addressMap.html

address_df = pd.read_csv("./Data/Archived/addressList.csv")
centerPointLat = 30.27444  #these are the coordinates of the Texas State Capitol building
centerPointLon = -97.74028 #these are the coordinates of the Texas State Capitol building
magnificationFactor = 10
pointColor = "blue"
pointSize = 100
mapOutputFile = "addressMap.html"
df = address_df

gmap = gmplot.GoogleMapPlotter(centerPointLat, centerPointLon, magnificationFactor)

gmap.scatter(df["lat"], df["lon"], pointColor, size=pointSize, marker=False)

gmap.draw("./Presentation/" + mapOutputFile)

len(df)

20177

## Zillow API Calls using Address and Zipcode

In [ ]:
#############################
##### VALERIE'S BLOCKS #####
###########################

# Function for reading in csv, checking for headers, and appending if appropriate
def csvDFappend(oldCSVfilepath, newColumn):
    csvIN = pd.read_csv(oldCSVfilepath)
    DF = pd.DataFrame(csvIN)
    # Checking to ensure new header name does not match any current headers
    colNames = DF.columns
    for value in colNames:
        if value == newColumn:
            print("Cannot append column that matches an existing column name")
            return DF
    # Check to ensure length of newColumn matches length of current dataframe columns
    if len(newColumn) != len(DF):
        print("Cannot append column that is not the same length as existing dataframe")
        return DF
    # Append newColumn to Dataframe
    DF[newColumn] = newColumn
    return DF

In [ ]:
### TEST BLCOK FOR FUNCTION ELEMENTS OF csvDFappend ###

colNames = masterDFIMPORTclean.columns
newCol = "zipcode"
for value in colNames:
    print(value)
    if value == newCol:
        print("Cannot append column that matches an existing column name")
        break
        
if len(address_df["zipcode"]) != len(masterDFIMPORTclean):
    print("Cannot append column that is not the same length as existing dataframe")  

In [ ]:
# Tiny sample to work with looping without exhausting API call limits
addressListTiny_csv = "./Data/addressListTiny.csv"

address_sample = pd.read_csv(addressListTiny_csv)
address_df = pd.DataFrame(address_sample)
print(len(address_df))

In [6]:
# Funtion for reading CSV in as DataFrame
def csvDF(oldCSVfilepath):
    csvIN = pd.read_csv(oldCSVfilepath)
    DF = pd.DataFrame(csvIN)
    return DF

# Function for converting DataFrame to CSV
def DFcsv(dataframe, newCSVfilepath):
    newCSVfilepath = pd.to_csv(dataframe)
    print(f"{dataframe} successfully written to {newCSVfilepath}")

In [8]:
##################################################################################################  EXPLORING katSampleData.csv
playDF = csvDF("../Data/Archived/katSampleData.csv")
playDF.columns


Index(['zid', 'address', 'zipcode', 'alat', 'alon', 'valuation',
       'value_change', 'value_range_high', 'value_range_low',
       'value_percentile', 'sqft', 'lot_sqft', 'year_built', 'bathrooms',
       'bedrooms', 'last_sold_date', 'last_sold_price'],
      dtype='object')

In [ ]:
columns2drop = ["value_percentile", "bedrooms", "bathrooms"]

In [9]:
playDF.sort_values("year_built")

,zid,address,zipcode,alat,alon,valuation,value_change,value_range_high,value_range_low,value_percentile,sqft,lot_sqft,year_built,bathrooms,bedrooms,last_sold_date,last_sold_price
264,70342502,2307 Kinney Road,78704,30.245655,-97.773194,823490.0,-10761.0,938779.0,782316.0,0,2450.0,16117.0,1904.0,NaN,4.0,NaN,NaN
190,29400584,3913 Avenue F,78751,30.302324,-97.730904,868539.0,-15928.0,1007505.0,781685.0,0,2450.0,10000.0,1906.0,NaN,4.0,NaN,NaN
147,29406697,5304 Bennett Avenue,78751,30.313371,-97.712326,386134.0,-9050.0,436331.0,332075.0,0,1122.0,9147.0,1918.0,NaN,2.0,06/21/2016,461500.0
68,29459138,1611 Nickerson Street,78704,30.246782,-97.749181,1008679.0,-46082.0,1099460.0,857377.0,0,2444.0,4452.0,1920.0,NaN,4.0,NaN,NaN
133,29329271,806 Theresa Avenue,78703,30.280340,-97.765907,667025.0,-22874.0,720387.0,620333.0,0,1348.0,6011.0,1923.0,NaN,3.0,03/10/2016,NaN
91,29398262,3504 Duval Street,78751,30.298054,-97.730640,504375.0,-29812.0,549769.0,464025.0,0,1100.0,8363.0,1924.0,NaN,2.0,NaN,NaN
184,62605359,103 Brazos Street,78701,30.265890,-97.741957,552673.0,-21267.0,784796.0,491879.0,0,1319.0,788.0,1925.0,NaN,1.0,NaN,NaN
74,29382543,1204 Taylor Street,78702,30.256762,-97.733412,454525.0,-2700.0,477251.0,431799.0,0,852.0,5749.0,1929.0,NaN,1.0,12/09/2015,NaN
31,122962433,6606 Hergotz Lane,78742,30.245467,-97.684546,286423.0,12931.0,326522.0,252052.0,0,1792.0,7406.0,1932.0,NaN,4.0,NaN,NaN
3,29400629,807 Park Boulevard,78751,30.301584,-97.722499,556665.0,-39191.0,629031.0,517698.0,0,1905.0,12685.0,1936.0,NaN,NaN,NaN,NaN


In [6]:
######################################################################################  MODIFIED TO CREATE SMALL SAMPLE FOR KAT
sampleDF = csvDF("./Data/Archived/addressListTiny.csv")

In [7]:
######################################################################################  MODIFIED TO CREATE SMALL SAMPLE FOR KAT
sampleDF.head()

,house #,street,zipcode,lat,lon,neighborhood
0,8109,Boggy Ridge Drive,78748,30.182018,-97.795088,Dittmar - Cooper
1,8507,Caspian Drive,78749,30.199637,-97.841057,Woodstone Village
2,950,Westbank Drive,78746,30.278182,-97.809151,Ledgeway
3,4208,Steck Avenue,78759,30.377557,-97.757560,Mesa Forest
4,807,Park Boulevard,78751,30.301572,-97.722516,Hyde Park


In [ ]:
addressDF = csvDF(addressList_csv)
print(len(addressDF))

In [13]:
######################################################################################  MODIFIED TO CREATE SMALL SAMPLE FOR KAT

############### LOOPING FUNCTION FULLY OPERATIONAL ################
####### HOWEVER, ZILLOW ONLY ALLOWS 1000 API CALLS PER DAY #######

# Zillow API call function using address and zipcode
def zCall(API, index, address, zipcode):
    APIkey = API[index]
    zillow_data = ZillowWrapper(APIkey)
    deep_search_response = zillow_data.get_deep_search_results(address, zipcode)
    result = GetDeepSearchResults(deep_search_response)
    return result


# List containers for collected property data
zid = []
addresses = []
alats = []
alons = []
valuation = []
valChange = []
valRhigh = []
valRlow = []
valPercent = []
sqft = []
lotsqft = []
yearBuilt = []
baths = []
beds = []
lastSold = []
lastPrice = []


# List of Zillow API keys to loop through due to daily API call limits
zAPIs = [Zapi, Zapi01, Zapi02, Zapi03, Zapi04, Zapi05, Zapi06, Zapi07, Zapi08, Zapi09, 
         Zapi10, Zapi11, Zapi12, Zapi13, Zapi14, Zapi15, Zapi16, Zapi17, Zapi18, Zapi19, 
         Zapi20, Ztroy1, Ztroy2, Ztroy3, Zseth, Zseth2, Zkat, Zval, Zyuta]
index = 0
    
for row, home in sampleDF.iterrows():
    address = str(sampleDF["house #"][row]) + " " + str(sampleDF["street"][row])
    addresses.append(address)
    zipcode = sampleDF["zipcode"][row]
    print(f"Processing {address}, {zipcode} (index {row}).")
    
    result = None
    try:
        try:
            result = zCall(zAPIs, index, address, zipcode)
            print(f"{row} Success!")
        except KeyError:  ### ERROR FOR API CALL LIMIT EXCEEDED ###
            print(f"KeyError has occurred for {address}, {zipcode} (index {row}).")
            index += 1
            print(f"Proceeding to API[{index}]")
            if index >= len(zAPIs):
                print(f"API[{index}] does not exist. Need more API keys to complete analysis.")
                break
            result = zCall(zAPIs, index, address, zipcode)

    except:
        print(f"No record found for {address}, {zipcode} (index {row}). Appending lists with null values")
        zid.append(None)
        alats.append(None)
        alons.append(None)
        valuation.append(None)
        valChange.append(None)
        valRhigh.append(None)
        valRlow.append(None)
        valPercent.append(None)
        sqft.append(None)
        lotsqft.append(None)
        yearBuilt.append(None)
        baths.append(None)
        beds.append(None)
        lastSold.append(None)
        lastPrice.append(None)
        continue

    try:
        zillowID = result.zillow_id
        zid.append(zillowID)
    except:
        print(f"No zid found for {address}, {zipcode} (index {row}). Appending list with null values")
        zid.append(None)

    try:
        alat = result.latitude
        alats.append(alat)
    except:
        print(f"No alat found for {address}, {zipcode} (index {row}). Appending list with null values")
        alats.append(None)

    try:
        alon = result.longitude
        alons.append(alon)
    except:
        print(f"No alon found for {address}, {zipcode} (index {row}). Appending list with null values")
        alons.append(None)

    try:    
        val = int(result.zestimate_amount)
        valuation.append(val)
    except:
        print(f"No valuation found for {address}, {zipcode} (index {row}). Appending list with null values")
        valuation.append(None)
        
    try:    
        change = result.zestimate_value_change
        valChange.append(change)
    except:
        print(f"No valuation change found for {address}, {zipcode} (index {row}). Appending list with null values")
        valChange.append(None)
        
    try:    
        high = result.zestimate_valuation_range_high
        valRhigh.append(high)
    except:
        print(f"No valuation range high found for {address}, {zipcode} (index {row}). Appending list with null values")
        valRhigh.append(None)

    try:    
        low = result.zestimate_valuationRange_low
        valRlow.append(low)
    except:
        print(f"No valuation range low found for {address}, {zipcode} (index {row}). Appending list with null values")
        valRlow.append(None)

    try:    
        percent = result.zestimate_percentile
        valPercent.append(percent)
    except:
        print(f"No valuation percentile found for {address}, {zipcode} (index {row}). Appending list with null values")
        valPercent.append(None)
        
    try:
        zsqft = result.home_size
        sqft.append(zsqft)
    except:
        print(f"No sqft found for {address}, {zipcode} (index {row}). Appending list with null values")
        sqft.append(None)
        
    try:
        lot = result.property_size
        lotsqft.append(lot)
    except:
        print(f"No lot size for {address}, {zipcode} (index {row}). Appending list with null values")
        lotsqft.append(None)
        
    try:
        year = int(result.year_built)
        yearBuilt.append(year)
    except:
        print(f"No year built for {address}, {zipcode} (index {row}). Appending list with null values")
        yearBuilt.append(None)

    try:
        bath = int(result.bathrooms)
        baths.append(bath)
    except:
        print(f"No bathrooms for {address}, {zipcode} (index {row}). Appending list with null values")
        baths.append(None)
        
    try:
        bed = int(result.bedrooms)
        beds.append(bed)
    except:
        print(f"No bedrooms for {address}, {zipcode} (index {row}). Appending list with null values")
        beds.append(None)
        
    try:
        last = result.last_sold_date
        lastSold.append(last)
    except:
        print(f"No last sold date for {address}, {zipcode} (index {row}). Appending list with null values")
        lastSold.append(None)
        
    try:
        price = result.last_sold_price
        lastPrice.append(price)
    except:
        print(f"No last sold price for {address}, {zipcode} (index {row}). Appending list with null values")
        lastPrice.append(None)


Processing 8109 Boggy Ridge Drive, 78748 (index 0).
KeyError has occurred for 8109 Boggy Ridge Drive, 78748 (index 0).
Proceeding to API[1]
No bathrooms for 8109 Boggy Ridge Drive, 78748 (index 0). Appending list with null values
Processing 8507 Caspian Drive, 78749 (index 1).
1 Success!
No bathrooms for 8507 Caspian Drive, 78749 (index 1). Appending list with null values
No bedrooms for 8507 Caspian Drive, 78749 (index 1). Appending list with null values
Processing 950 Westbank Drive, 78746 (index 2).
2 Success!
No year built for 950 Westbank Drive, 78746 (index 2). Appending list with null values
No bathrooms for 950 Westbank Drive, 78746 (index 2). Appending list with null values
Processing 4208 Steck Avenue, 78759 (index 3).
No record found for 4208 Steck Avenue, 78759 (index 3). Appending lists with null values
Processing 807 Park Boulevard, 78751 (index 4).
4 Success!
No bathrooms for 807 Park Boulevard, 78751 (index 4). Appending list with null values
No bedrooms for 807 Park Bo

44 Success!
No bathrooms for 4705 Avenue F, 78751 (index 44). Appending list with null values
Processing 3401 Oak Springs Drive, 78721 (index 45).
45 Success!
No bathrooms for 3401 Oak Springs Drive, 78721 (index 45). Appending list with null values
Processing 1515 Capital of Texas Highway, 78746 (index 46).
No record found for 1515 Capital of Texas Highway, 78746 (index 46). Appending lists with null values
Processing 5804 Wier Hills Road, 78735 (index 47).
47 Success!
No bathrooms for 5804 Wier Hills Road, 78735 (index 47). Appending list with null values
No bedrooms for 5804 Wier Hills Road, 78735 (index 47). Appending list with null values
Processing 5907 Blanco River Pass, 78749 (index 48).
48 Success!
No bathrooms for 5907 Blanco River Pass, 78749 (index 48). Appending list with null values
Processing 9009 Wintercreeper Cove, 78735 (index 49).
49 Success!
No bathrooms for 9009 Wintercreeper Cove, 78735 (index 49). Appending list with null values
Processing 4300 Eskew Drive, 78749

No record found for 1324 Frontier Valley Drive, 78741 (index 91). Appending lists with null values
Processing 3607 Fall Trail, 78731 (index 92).
No record found for 3607 Fall Trail, 78731 (index 92). Appending lists with null values
Processing 9807 Brown Lane, 78754 (index 93).
No record found for 9807 Brown Lane, 78754 (index 93). Appending lists with null values
Processing 9310 Burleson Road, 78719 (index 94).
No record found for 9310 Burleson Road, 78719 (index 94). Appending lists with null values
Processing 6001 Sandshof Drive, 78724 (index 95).
95 Success!
No bathrooms for 6001 Sandshof Drive, 78724 (index 95). Appending list with null values
Processing 9508 Presa Abajo Drive, 78733 (index 96).
No record found for 9508 Presa Abajo Drive, 78733 (index 96). Appending lists with null values
Processing 9706 Saugus Lane, 78733 (index 97).
97 Success!
No bathrooms for 9706 Saugus Lane, 78733 (index 97). Appending list with null values
No bedrooms for 9706 Saugus Lane, 78733 (index 97).

No record found for 8360 Brodie Lane, 78745 (index 137). Appending lists with null values
Processing 5439 Spicewood Springs Road, 78759 (index 138).
No record found for 5439 Spicewood Springs Road, 78759 (index 138). Appending lists with null values
Processing 5104 Beverly Hills Drive, 78731 (index 139).
139 Success!
No bathrooms for 5104 Beverly Hills Drive, 78731 (index 139). Appending list with null values
Processing 4805 Emma Browning Avenue, 78617 (index 140).
No record found for 4805 Emma Browning Avenue, 78617 (index 140). Appending lists with null values
Processing 3504 Duval Street, 78751 (index 141).
141 Success!
No bathrooms for 3504 Duval Street, 78751 (index 141). Appending list with null values
Processing 402 Oak Ridge Drive, 78746 (index 142).
142 Success!
No bathrooms for 402 Oak Ridge Drive, 78746 (index 142). Appending list with null values
Processing 1416 Webberville Road, 78721 (index 143).
143 Success!
No bathrooms for 1416 Webberville Road, 78721 (index 143). Appe

182 Success!
No valuation found for 125 Brandon Way, 78733 (index 182). Appending list with null values
No bathrooms for 125 Brandon Way, 78733 (index 182). Appending list with null values
Processing 8604 Georgian Drive, 78753 (index 183).
183 Success!
No bathrooms for 8604 Georgian Drive, 78753 (index 183). Appending list with null values
Processing 5706 Eagle Cliff, 78731 (index 184).
184 Success!
No bathrooms for 5706 Eagle Cliff, 78731 (index 184). Appending list with null values
No bedrooms for 5706 Eagle Cliff, 78731 (index 184). Appending list with null values
Processing 3620 Ranch Creek Drive, 78730 (index 185).
185 Success!
No bathrooms for 3620 Ranch Creek Drive, 78730 (index 185). Appending list with null values
Processing nan San Marcos Street, 78767 (index 186).
No record found for nan San Marcos Street, 78767 (index 186). Appending lists with null values
Processing 4514 Rimrock Trail, 78723 (index 187).
187 Success!
No bathrooms for 4514 Rimrock Trail, 78723 (index 187). 

227 Success!
No bathrooms for 5752 North Scout Island Circle, 78731 (index 227). Appending list with null values
Processing 6600 West William Cannon Drive, 78735 (index 228).
No record found for 6600 West William Cannon Drive, 78735 (index 228). Appending lists with null values
Processing 713 Kinney Avenue, 78704 (index 229).
229 Success!
No bathrooms for 713 Kinney Avenue, 78704 (index 229). Appending list with null values
Processing 5600 Decker Lane, 78724 (index 230).
No record found for 5600 Decker Lane, 78724 (index 230). Appending lists with null values
Processing 1707 Overhill Drive, 78721 (index 231).
231 Success!
No bathrooms for 1707 Overhill Drive, 78721 (index 231). Appending list with null values
No bedrooms for 1707 Overhill Drive, 78721 (index 231). Appending list with null values
Processing 2412 Burleson Court, 78741 (index 232).
232 Success!
No bathrooms for 2412 Burleson Court, 78741 (index 232). Appending list with null values
Processing 5634 Sedona Drive, 78759 (ind

271 Success!
No bathrooms for 2417 Hartford Road, 78703 (index 271). Appending list with null values
Processing 5 Stegner Lane, 78746 (index 272).
272 Success!
No bathrooms for 5 Stegner Lane, 78746 (index 272). Appending list with null values
Processing 2921 Farm-to-Market Road 973, 78725 (index 273).
No record found for 2921 Farm-to-Market Road 973, 78725 (index 273). Appending lists with null values
Processing 2155 Parkway, 78703 (index 274).
No record found for 2155 Parkway, 78703 (index 274). Appending lists with null values
Processing 11012 Sierra Verde Trail, 78759 (index 275).
275 Success!
No bathrooms for 11012 Sierra Verde Trail, 78759 (index 275). Appending list with null values
No bedrooms for 11012 Sierra Verde Trail, 78759 (index 275). Appending list with null values
Processing 1209 Dusky Thrush Trail, 78746 (index 276).
276 Success!
No bathrooms for 1209 Dusky Thrush Trail, 78746 (index 276). Appending list with null values
No bedrooms for 1209 Dusky Thrush Trail, 78746 

Processing 8315 1/2 Steamline Circle, 78745 (index 315).
No record found for 8315 1/2 Steamline Circle, 78745 (index 315). Appending lists with null values
Processing 2712 Bartons Bluff Lane, 78746 (index 316).
316 Success!
No bathrooms for 2712 Bartons Bluff Lane, 78746 (index 316). Appending list with null values
Processing 2411 Campden Drive, 78745 (index 317).
317 Success!
No bathrooms for 2411 Campden Drive, 78745 (index 317). Appending list with null values
Processing 7728 FM 1625, 78744 (index 318).
No record found for 7728 FM 1625, 78744 (index 318). Appending lists with null values
Processing 1610 Dungan Lane, 78754 (index 319).
No record found for 1610 Dungan Lane, 78754 (index 319). Appending lists with null values
Processing 4208 Kings Court, 78724 (index 320).
320 Success!
No bathrooms for 4208 Kings Court, 78724 (index 320). Appending list with null values
No bedrooms for 4208 Kings Court, 78724 (index 320). Appending list with null values
Processing 8297 Decker Lane, 787

360 Success!
No bathrooms for 1120 Elder Circle, 78733 (index 360). Appending list with null values
Processing 1100 Fallwell Lane, 78617 (index 361).
No record found for 1100 Fallwell Lane, 78617 (index 361). Appending lists with null values
Processing 6414 Cottonmouth School Road, 78744 (index 362).
No record found for 6414 Cottonmouth School Road, 78744 (index 362). Appending lists with null values
Processing 1210 Bob Harrison Street, 78702 (index 363).
363 Success!
No bathrooms for 1210 Bob Harrison Street, 78702 (index 363). Appending list with null values
Processing 5301 East Oltorf Street, 78741 (index 364).
No record found for 5301 East Oltorf Street, 78741 (index 364). Appending lists with null values
Processing 7900 Loyola Lane, 78724 (index 365).
No record found for 7900 Loyola Lane, 78724 (index 365). Appending lists with null values
Processing 1940 Webberville Road, 78721 (index 366).
366 Success!
No bathrooms for 1940 Webberville Road, 78721 (index 366). Appending list wit

401 Success!
No bathrooms for 9405 Bell Mountain Drive, 78730 (index 401). Appending list with null values
Processing 2307 Kinney Road, 78704 (index 402).
402 Success!
No bathrooms for 2307 Kinney Road, 78704 (index 402). Appending list with null values
Processing 9500 Decker Lane, 78724 (index 403).
No record found for 9500 Decker Lane, 78724 (index 403). Appending lists with null values
Processing 9310 Burleson Road, 78719 (index 404).
No record found for 9310 Burleson Road, 78719 (index 404). Appending lists with null values
Processing 4722 Palisade Drive, 78731 (index 405).
405 Success!
No bathrooms for 4722 Palisade Drive, 78731 (index 405). Appending list with null values
Processing 6506 Auburnhill Street, 78723 (index 406).
406 Success!
No bathrooms for 6506 Auburnhill Street, 78723 (index 406). Appending list with null values
Processing 2300 Rosewood Avenue, 78702 (index 407).
No record found for 2300 Rosewood Avenue, 78702 (index 407). Appending lists with null values
Processi

In [14]:
######################################################################################  MODIFIED TO CREATE SMALL SAMPLE FOR KAT

# Checking to ensure lists are appropriate lengths
print(len(zid))
print(len(addresses))
print(len(alats))
print(len(alons))
print(len(valuation))
print(len(valChange))
print(len(valRhigh))
print(len(valRlow))
print(len(valPercent))
print(len(sqft))
print(len(lotsqft))
print(len(yearBuilt))
print(len(baths))
print(len(beds))
print(len(lastSold))
print(len(lastPrice))

# Referring back to addressList_csv generated dataframe for relevant info
sampleDF.head()

442
442
442
442
442
442
442
442
442
442
442
442
442
442
442
442


,house #,street,zipcode,lat,lon,neighborhood
0,8109,Boggy Ridge Drive,78748,30.182018,-97.795088,Dittmar - Cooper
1,8507,Caspian Drive,78749,30.199637,-97.841057,Woodstone Village
2,950,Westbank Drive,78746,30.278182,-97.809151,Ledgeway
3,4208,Steck Avenue,78759,30.377557,-97.757560,Mesa Forest
4,807,Park Boulevard,78751,30.301572,-97.722516,Hyde Park


In [15]:
######################################################################################  MODIFIED TO CREATE SMALL SAMPLE FOR KAT

katSampleDF = pd.DataFrame({
    "zid": zid,
    "address": addresses,
    "zipcode": sampleDF["zipcode"],
    "alat": alats,
    "alon": alons,
    "valuation": valuation,
    "value_change": valChange,
    "value_range_high": valRhigh,
    "value_range_low": valRlow,
    "value_percentile": valPercent,
    "sqft": sqft,
#     "value sqft": valsqft,
    "lot_sqft": lotsqft,
    "year_built": yearBuilt,
    "bathrooms": baths,
    "bedrooms": beds,
    "last_sold_date": lastSold,
    "last_sold_price": lastPrice,
})

print(len(katSampleDF))
katSampleDF.head(10)

442


,zid,address,zipcode,alat,alon,valuation,value_change,value_range_high,value_range_low,value_percentile,sqft,lot_sqft,year_built,bathrooms,bedrooms,last_sold_date,last_sold_price
0,58316011,8109 Boggy Ridge Drive,78748,30.182015,-97.795063,278601.0,9703,292531,264671,0,1476,6003,1979.0,None,3.0,None,None
1,29498766,8507 Caspian Drive,78749,30.199658,-97.841024,313492.0,-1374,329167,297817,0,1654,15141,1983.0,None,NaN,None,None
2,2132336893,950 Westbank Drive,78746,30.278708,-97.808911,848997.0,27409,1256516,687688,0,1582,None,NaN,None,3.0,None,None
3,None,4208 Steck Avenue,78759,None,None,NaN,None,None,None,None,None,None,NaN,None,NaN,None,None
4,29400629,807 Park Boulevard,78751,30.301584,-97.722499,556665.0,-39191,629031,517698,0,1905,12685,1936.0,None,NaN,None,None
5,None,10109 Slaughter Creek Drive,78748,None,None,NaN,None,None,None,None,None,None,NaN,None,NaN,None,None
6,29462460,601 Jet Lane,78742,30.235276,-97.677214,258682.0,3979,276790,240574,0,1015,161128,1945.0,None,NaN,None,None
7,29349645,4409 Jessamine Hollow,78731,30.348537,-97.776564,519926.0,949,623911,483531,0,2900,5149,1977.0,None,NaN,None,None
8,None,8800 Daffan Lane,78724,None,None,NaN,None,None,None,None,None,None,NaN,None,NaN,None,None
9,29343151,4501 Ridge Oak Drive,78731,30.329848,-97.769843,2625759.0,-178712,3649805,2415698,0,10778,45011,1960.0,None,NaN,None,None


In [22]:
######################################################################################  MODIFIED TO CREATE SMALL SAMPLE FOR KAT

katDFclean = katSampleDF.dropna(how="any", subset=["zid"])
len(katDFclean)

285

In [25]:
######################################################################################  MODIFIED TO CREATE SMALL SAMPLE FOR KAT

DFcsv(katDFclean, "./Data/Archived/katSampleData.csv")
katDFclean.head(30)

Successfully written to './Data/Archived/katSampleData.csv'


,zid,address,zipcode,alat,alon,valuation,value_change,value_range_high,value_range_low,value_percentile,sqft,lot_sqft,year_built,bathrooms,bedrooms,last_sold_date,last_sold_price
0,58316011,8109 Boggy Ridge Drive,78748,30.182015,-97.795063,278601.0,9703,292531,264671,0,1476,6003,1979.0,None,3.0,None,None
1,29498766,8507 Caspian Drive,78749,30.199658,-97.841024,313492.0,-1374,329167,297817,0,1654,15141,1983.0,None,NaN,None,None
2,2132336893,950 Westbank Drive,78746,30.278708,-97.808911,848997.0,27409,1256516,687688,0,1582,None,NaN,None,3.0,None,None
4,29400629,807 Park Boulevard,78751,30.301584,-97.722499,556665.0,-39191,629031,517698,0,1905,12685,1936.0,None,NaN,None,None
6,29462460,601 Jet Lane,78742,30.235276,-97.677214,258682.0,3979,276790,240574,0,1015,161128,1945.0,None,NaN,None,None
7,29349645,4409 Jessamine Hollow,78731,30.348537,-97.776564,519926.0,949,623911,483531,0,2900,5149,1977.0,None,NaN,None,None
9,29343151,4501 Ridge Oak Drive,78731,30.329848,-97.769843,2625759.0,-178712,3649805,2415698,0,10778,45011,1960.0,None,NaN,None,None
12,70339207,11201 Long Summer Drive,78754,30.359344,-97.640621,297039.0,3970,311891,282187,0,3183,7867,2006.0,None,NaN,None,None
13,58302468,504 East Braker Lane,78753,30.378232,-97.67872,254661.0,-3921,277580,236835,0,1264,9500,1959.0,None,NaN,None,None
14,29402786,5703 Coolbrook Drive,78724,30.304843,-97.656442,360639.0,-2969,385884,339001,0,2282,9936,1972.0,None,NaN,None,None


In [ ]:
############### LOOPING FUNCTION FULLY OPERATIONAL ################
####### HOWEVER, ZILLOW ONLY ALLOWS 1000 API CALLS PER DAY #######

# Zillow API call function
def zCall(API, index, address, zipcode):
    APIkey = API[index]
    zillow_data = ZillowWrapper(APIkey)
    deep_search_response = zillow_data.get_deep_search_results(address, zipcode)
    result = GetDeepSearchResults(deep_search_response)
    return result

# List containers for collected property data
zid = []
addresses = []
alats = []
alons = []
valuation = []
sqft = []

# List of Zillow API keys to loop through due to daily API call limits
zAPIs = [Zapi, Zapi01, Zapi02, Zapi03, Zapi04, Zapi05, Zapi06, Zapi07, Zapi08, Zapi09, 
         Zapi10, Zapi11, Zapi12, Zapi13, Zapi14, Zapi15, Zapi16, Zapi17, Zapi18, Zapi19, 
         Zapi20, Ztroy1, Ztroy2, Ztroy3, Zseth, Zseth2, Zkat, Zval, Zyuta]
index = 0
    
for row, home in addressDF.iterrows():
    address = str(addressDF["house #"][row]) + " " + str(addressDF["street"][row])
    addresses.append(address)
    zipcode = addressDF["zipcode"][row]
    print(f"Processing {address}, {zipcode} (index {row}).")
    
    result = None
    try:
        try:
            result = zCall(zAPIs, index, address, zipcode)
            print(f"{row} Success!")
        except KeyError:  ### ERROR FOR API CALL LIMIT EXCEEDED ###
            print(f"KeyError has occurred for {address}, {zipcode} (index {row}).")
            index += 1
            print(f"Proceeding to API[{index}]")
            if index >= len(zAPIs):
                print(f"API[{index}] does not exist. Need more API keys to complete analysis.")
                break
            result = zCall(zAPIs, index, address, zipcode)

    except:
        print(f"No record found for {address}, {zipcode} (index {row}). Appending lists with null values")
        zid.append(None)
        alats.append(None)
        alons.append(None)
        valuation.append(None)
        sqft.append(None)
        continue

    try:
        zillowID = result.zillow_id
        zid.append(zillowID)
    except:
        print(f"No zid found for {address}, {zipcode} (index {row}). Appending list with null values")
        zid.append(None)

    try:
        alat = result.latitude
        alats.append(alat)
    except:
        print(f"No alat found for {address}, {zipcode} (index {row}). Appending list with null values")
        alats.append(None)

    try:
        alon = result.longitude
        alons.append(alon)
    except:
        print(f"No alon found for {address}, {zipcode} (index {row}). Appending list with null values")
        alons.append(None)

    try:    
        val = int(result.zestimate_amount)
        valuation.append(val)
    except:
        print(f"No valuation found for {address}, {zipcode} (index {row}). Appending list with null values")
        valuation.append(None)

    try:
        zsqft = int(result.home_size)
        sqft.append(zsqft)
    except:
        print(f"No sqft found for {address}, {zipcode} (index {row}). Appending list with null values")
        sqft.append(None)

In [ ]:
print(len(addresses))

## Master Dataframe Creation

In [ ]:
# Checking to ensure lists are appropriate lengths
print(len(zid))
print(len(alats))
print(len(alons))
print(len(addresses))
print(len(valuation))
print(len(sqft))
print(len(valsqft))

# Referring back to addressList_csv generated dataframe for relevant info
addressDF.head()

In [ ]:
# FULL SAMPLE FROM 40,000 RANDOM LAT, LON GENERATION

masterDF = pd.DataFrame({
    "Zillow ID": zid,
    "address": addresses,
    "zipcode": addressDF["zipcode"],
    "alat": alats,
    "alon": alons,
    "valuation": valuation,
    "sqft": sqft,
#     "value sqft": valsqft,
    "neighborhood": addressDF["neighborhood"],
})
print(len(masterDF))
masterDF.head(10)

In [ ]:
masterDFclean = masterDF.dropna(how="any", subset=["Zillow ID"])
len(masterDFclean)

In [ ]:
# masterDF to csv
masterDFclean.to_csv(masterData_csv, index=False, header=True)
masterDFclean.head(30)

## Zillow Data Cleaning

In [ ]:
masterDF = csvDF(masterData_csv)
print(len(masterDF))
masterDF.head()

In [ ]:
# DATA CLEANING #
# Dropping duplicates
masterDFdrops = masterDF.drop_duplicates(subset=["Zillow ID"], keep="first")
print(len(masterDFdrops))
masterDFdrops.head()

In [ ]:
# Cleaning out None values for "valuation"
masterDFdrops = masterDFdrops.dropna(how="any", subset=["valuation"])
print(len(masterDFdrops))
masterDFdrops.head()

In [ ]:
# Cleaning out None values for "sqft" 
masterDFdrops = masterDFdrops.dropna(how="any", subset=["sqft"])
print(len(masterDFdrops))
masterDFdrops.head()

### Calculate Value per Sqft

In [ ]:
# Calculate "value sqft" after None value rows removed for "valuation" and "sqft"

valsqft = []
for row, value in masterDFdrops.iterrows():
    try:
        vsqft = round((masterDFdrops["valuation"][row] / masterDFdrops["sqft"][row]), 2)
        valsqft.append(vsqft)
    except: ### THIS ERROR SHOULD NO LONGER PRINT BECAUSE NONE VALUES WERE PREVIOUSLY REMOVED
        print("Cannot perform math with NoneType")
        valsqft.append(None)

In [ ]:
# Checking to ensure lists are appropriate lengths
print(len(masterDFdrops))
print(len(valsqft))

In [ ]:
# Adding "value sqft" column
masterDFdrops["value sqft"] = valsqft

# Reordering columns
masterDFdrops = masterDFdrops[['Zillow ID', 'address', 'zipcode', 'alat', 'alon', 'valuation', 'sqft', 
                               'value sqft', 'neighborhood', 'tractCode', 'countyFips', 'stateFips', 'commuteTime']]
masterDFdrops.head()

In [ ]:
DFcsv(masterDFdrops, masterDFIMPORTclean_csv)

In [ ]:
masterDFcleaning = csvDF(masterDFIMPORTclean_csv)
masterDFcleaning.head()

In [ ]:
# Function for finding and dropping rows with nonsense values (e.g. valuation > $10,000,000)
def dropNonsense(dataframe, columnName, minVal, maxVal):
    dropIndices = []
    for index, row in dataframe.iterrows():
        val = dataframe[columnName][index]
        if (maxVal is not None) and (val > maxVal) or (minVal is not None) and (val < minVal):
            dropIndices.append(index)
    return dataframe.drop(index=dropIndices)

In [ ]:
# Establish reasonable values for columns
reasonableVals = [
    ("valuation", 20000, 10000000),
    ("sqft", 500, 10000),
    ("value sqft", 0, 1500)
]

# Loop through dataframe to drop nonsense data
masterDFcleaning = masterDFcleaning
for entry in reasonableVals:
    masterDFcleaning = dropNonsense(masterDFcleaning, entry[0], entry[1], entry[2])

# Confirm count
len(masterDFcleaning)

In [ ]:
# Checking high and low values with sort
masterDFcleaning.sort_values(by="value sqft", ascending=False)

In [ ]:
print(len(masterDFcleaning))
DFcsv(masterDFcleaning, masterDataCLEAN_csv)

### Merging Master Data with Commute Times

In [ ]:
commuteDF = csvDF("./Data/masterData.csv")
commuteDF.head()
print(len(commuteDF))
print(len(masterDFcleaning))
print("Commute")
print(commuteDF.dtypes)
print("Master")
print(masterDFcleaning.dtypes)
commuteDF.head()


In [ ]:
MASTERdf = pd.merge(masterDFcleaning, 
                    commuteDF[['tractCode', 'countyFips', 'stateFips', 'commuteTime']], 
                    how="left",
                    on='Zillow ID')

In [ ]:
print(len(MASTERdf))
MASTERdf.head()

## Plot Zillow Matched Addresses on a Map

In [18]:
# This cell by Troy
# This section will plot points on a Google map centered at centerPointLat and centerPointLon with a magnification of magFactor
# It plots the Zillow addresses we have matched from our randomly selected 
# The resulting map is saved to a file called addressMap.html

masterData_df = pd.read_csv("./Data/masterDataCLEAN.csv")
centerPointLat = 30.27444  #these are the coordinates of the Texas State Capitol building
centerPointLon = -97.74028 #these are the coordinates of the Texas State Capitol building
magnificationFactor = 10
pointColor = "green"
pointSize = 100
mapOutputFile = "masterDataMap.html"
df = masterData_df

gmap = gmplot.GoogleMapPlotter(centerPointLat, centerPointLon, magnificationFactor)

gmap.scatter(df["alat"], df["alon"], pointColor, size=pointSize, marker=False)

gmap.draw("./Presentation/" + mapOutputFile)

len(df)

8530

## Crime Data

In [ ]:
# Kat's section

## School Data

In [ ]:
# Seth's section

##  Commute Time Data

In [ ]:
# Troy's section

# This section reads the masterData csv file and uses the lat lon coordinates to get the us census tract code for that address
# It then uses the tract code to access the average commute times reported for that tract


# Load in data frame from file with lat and lon
masterData_df = pd.read_csv(masterData_csv)


lats = masterData_df['alat']
lons = masterData_df['alon']

# Set up arrays for new data
tractCodeList = []
countyFipsList = []
stateFipsList = []


# Use us census API to get state fips, county fips, and tract code for all addr in dataframe
for lat, lon in zip(lats,lons):
    print("getting data for " + str(lat) + str(lon))
    targetUrl = "https://geocoding.geo.census.gov/geocoder/geographies/coordinates?x=" + str(lon) + "&y=" + str(lat) + "&benchmark=Public_AR_Census2010&vintage=Census2010_Census2010&layers=14&format=json"
    results = requests.get(targetUrl).json()
    print(results)
    tractCodeList.append(results["result"]["geographies"]["Census Blocks"][0]["TRACT"])
    countyFipsList.append(results["result"]["geographies"]["Census Blocks"][0]["COUNTY"])
    stateFipsList.append(results["result"]["geographies"]["Census Blocks"][0]["STATE"])
    
# Load new data into masterData
masterData_df["tractCode"] = tractCodeList
masterData_df["countyFips"] = countyFipsList
masterData_df["stateFips"] = stateFipsList

masterData_df.head()

In [ ]:
masterData_df.to_csv(masterData_csv, index=False, header=True)

In [ ]:
# This cell uses us census state fips, county fips, and tract code to access commute times


# Set up list to hold new data
commuteTimeList = []

loopCounter = 0

# Use us census API to get commute time from state, county and tract
for state, county, tract in zip(stateFipsList, countyFipsList ,tractCodeList):
    print("getting data for " + state + county + tract)
    loopCounter += 1
    print(loopCounter)
    targetUrl = "https://api.census.gov/data/2016/acs/acs5/profile?get=DP03_0025E,NAME&for=tract:" + tract + "&in=state:" + state + " county:" + county + "&key=" + census_API_Key
    results = requests.get(targetUrl).json()
    #print(results)
    commuteTimeList.append(results[1][0])

# Add commute time to masterData_df
masterData_df["commuteTime"] = commuteTimeList

# write a CSV
#masterData_df.to_csv(masterData_csv, index=False, header=True)

masterData_df.head()



In [ ]:
masterData_df.to_csv(masterData_csv, index=False, header=True)

## Heat Mapper

In [ ]:
# Troy's section


gmaps.configure(api_key=google_API_Key)

In [ ]:
# This cell creates a test masterData_df by pulling in Yuta's address file and adds a column as a testm "value to map"
# This cell can be deleted as soon as there is a master data file that includes a property value column or some other value to plot
# The last digit of the zipcode is used as a value that will vary by area and a random number between 0 and 1 is added to create variation in the weights

masterData_df = pd.read_csv(addressList_csv)
zips = masterData_df["zipcode"]
valueToMap = []

for zip in zips:
    lastDigit = zip[-1:]
#    print(last2Digits)
    valueToMap.append(int(lastDigit) + random.uniform(0.0,1.0))
    
masterData_df["valueToMap"] = valueToMap
masterData_df.head()

In [ ]:
# This cell uses gmaps library to create a google heat map from the data in a master data file.
# The masterData csv file is taken as input
# The lat and lon columns are taken as the coordinates for hte heatmap 
# The user specified column is taken as the weighting valies fo each coordinate point

df = masterData_df
columnToMap = 'valueToMap'
max_intensity = df[columnToMap].max()

fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(df[['lat', 'lon']], weights=df[columnToMap], max_intensity=max_intensity, point_radius=10.0)
fig.add_layer(heatmap_layer)
fig

In [ ]:
# this is a function version of the cell above
# the function takes columnToMap as the weights for the points defined by 'lat' and 'lon' columns in the dataframe
# the dataframe can be included as a parameter, if it is not included masterData_df is assumed

def heatMapper(columnToMap, df = masterData_df):
    
    max_intensity = df[columnToMap].max()
    
    fig = gmaps.figure()
    heatmap_layer = gmaps.heatmap_layer(df[['lat', 'lon']], weights=df[columnToMap], max_intensity=max_intensity, point_radius=10.0)
    fig.add_layer(heatmap_layer)

    return;

In [ ]:
heatMapper(columnToMap = 'valueToMap')
fig